# Environment

In [1]:
!python -V

Python 3.10.8


In [2]:
!pip install plotly
!pip install Prophet

In [3]:
import os
import re
import warnings

import plotly.express as px
import numpy as np

import pyspark
import pyspark.pandas as ps

from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

In [4]:
ps.options.display.max_rows = 10

ps.set_option('plotting.backend', 'plotly')

os.environ["PYARROW_IGNORE_TIMEZONE"] = "1"
os.environ["PYSPARK_DRIVER_PYTHON_OPTS"] = "lab"

warnings.filterwarnings("ignore") 

In [5]:
print(f'pyspark version: {pyspark.__version__}')

pyspark version: 3.3.1


# Spark Secion

In [6]:
conf = pyspark.SparkConf()

conf.setAppName('Task1')
conf.setMaster('local[2]')

sc = SparkContext.getOrCreate(conf)
spark = SparkSession(sc)

# Load Data

In [7]:
wd = os.getcwd()
path_data = os.path.join(wd, 'study_case/Task1/data/', 'forcasting_cs_data.csv')
#path_data = os.path.join(wd, 'data', 'forcasting_cs_data.csv')
df = ps.read_csv(path_data)
df.head()

,Product,date,Sales,Price Discount (%),In-Store Promo,Catalogue Promo,Store End Promo,Google_Mobility,Covid_Flag,V_DAY,EASTER,CHRISTMAS
0,SKU1,05/02/17,27750,0%,0,0,0,0.0,0,0,0,0
1,SKU1,12/02/17,29023,0%,1,0,1,0.0,0,1,0,0
2,SKU1,19/02/17,45630,17%,0,0,0,0.0,0,0,0,0
3,SKU1,26/02/17,26789,0%,1,0,1,0.0,0,0,0,0
4,SKU1,05/03/17,41999,17%,0,0,0,0.0,0,0,0,0


## Column name homogenization

In [8]:
columns = [re.sub("[^A-Z0-9 _]", "", column, 0, re.IGNORECASE) for column in df.columns]
columns = [column.lower().replace(' ','_') for column in columns]
columns = ['product',
            'date',
            'sales',
            'price_discount',
            'instore_promo',
            'catalogue_promo',
            'store_end_promo',
            'google_mobility',
            'covid_flag',
            'v_day',
            'easter',
            'christmas']
df.columns = columns

In [21]:
products = np.sort(df['product'].unique().values)
for product in products:
    df[df['product']==product].plot.line(x='date', y='sales', color="product", title=f"{product} sales over 2017-2020").show()

## Type convertion

In [ ]:
df.dtypes

product             object
date                object
sales                int32
price_discount      object
instore_promo        int32
catalogue_promo      int32
store_end_promo      int32
google_mobility    float64
covid_flag           int32
v_day                int32
easter               int32
christmas            int32
dtype: object

In [ ]:
df['date'] = ps.to_datetime(df['date'], infer_datetime_format=True)
df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month
df['cw'] = df['date'].dt.week
df['quarter'] = df['date'].dt.quarter

In [ ]:
df['product_numerical'] = df['product'].apply(lambda x: int(x.replace('SKU','')))
df['price_discount'] = df['price_discount'].apply(lambda x: float(x.replace('%','')))

## Shape and missing values 

In [ ]:
df.shape

(1218, 17)

In [ ]:
df.isna().sum()

product            0
date               0
sales              0
price_discount     0
instore_promo      0
catalogue_promo    0
store_end_promo    0
google_mobility    0
covid_flag         0
v_day              0
dtype: int64
Showing only the first 10

In [ ]:
df.isnull().sum()

product            0
date               0
sales              0
price_discount     0
instore_promo      0
catalogue_promo    0
store_end_promo    0
google_mobility    0
covid_flag         0
v_day              0
dtype: int64
Showing only the first 10

# Exploration

## Stats

In [ ]:
df.head()

,product,date,sales,price_discount,instore_promo,catalogue_promo,store_end_promo,google_mobility,covid_flag,v_day,easter,christmas,year,month,cw,quarter,product_numerical
0,SKU1,2017-05-02,27750,0.0,0,0,0,0.0,0,0,0,0,2017,5,18,2,1
1,SKU1,2017-12-02,29023,0.0,1,0,1,0.0,0,1,0,0,2017,12,48,4,1
2,SKU1,2017-02-19,45630,17.0,0,0,0,0.0,0,0,0,0,2017,2,7,1,1
3,SKU1,2017-02-26,26789,0.0,1,0,1,0.0,0,0,0,0,2017,2,8,1,1
4,SKU1,2017-05-03,41999,17.0,0,0,0,0.0,0,0,0,0,2017,5,18,2,1


In [ ]:
df.describe()

,date,sales,price_discount,instore_promo,catalogue_promo,store_end_promo,google_mobility,covid_flag,v_day,easter,christmas,year,month,cw,quarter,product_numerical
count,1218,1218.000000,1218.000000,1218.000000,1218.000000,1218.000000,1218.000000,1218.000000,1218.00000,1218.00000,1218.000000,1218.000000,1218.000000,1218.000000,1218.000000,1218.000000
mean,2019-01-11 22:20:41.379310,30294.000000,25.104269,0.000000,0.000000,0.000000,-2.377406,0.000000,0.00000,0.00000,0.000000,2018.000000,6.000000,26.000000,2.000000,3.000000
min,2017-01-10 00:00:00,0.000000,0.000000,0.000000,0.000000,0.000000,-28.490000,0.000000,0.00000,0.00000,0.000000,2017.000000,1.000000,1.000000,1.000000,1.000000
25%,2018-01-14 00:00:00,7212.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,0.000000,2018.000000,4.000000,15.000000,2.000000,2.000000
50%,2019-01-12 00:00:00,19735.000000,25.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,0.000000,2019.000000,7.000000,27.000000,3.000000,3.000000
75%,2020-01-03 00:00:00,40295.000000,40.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.00000,0.00000,0.000000,2020.000000,10.000000,40.000000,4.000000,5.000000
max,2020-12-27 00:00:00,288322.000000,83.000000,1.000000,1.000000,1.000000,3.900000,1.000000,1.00000,1.00000,1.000000,2020.000000,12.000000,52.000000,4.000000,6.000000
std,None,35032.527297,21.549350,0.499425,0.409346,0.476828,5.806291,0.418804,0.13904,0.13904,0.136169,1.107197,3.423686,14.939332,1.107488,1.703666


In [ ]:
df.corr()

,sales,price_discount,instore_promo,catalogue_promo,store_end_promo,google_mobility,covid_flag,v_day,easter,christmas,year,month,cw,quarter,product_numerical
sales,1.000000,0.403196,0.240891,-0.121081,0.235439,0.058781,-0.085571,-0.008442,-0.010266,-0.035916,-0.078891,0.059806,0.059702,0.063602,-0.085193
price_discount,0.403196,1.000000,0.217904,-0.074055,0.216127,-0.213534,0.281537,-0.044565,0.000959,-0.012153,0.250843,0.050672,0.057329,0.040958,0.279167
instore_promo,0.240891,0.217904,1.000000,-0.491438,0.367004,0.056074,-0.024732,0.019761,0.019761,0.013799,-0.031572,0.043007,0.044705,0.033147,0.021807
catalogue_promo,-0.121081,-0.074055,-0.491438,1.000000,0.116301,0.073260,-0.094373,-0.044805,-0.044805,0.045834,-0.038510,-0.014503,-0.017327,-0.000896,0.113924
store_end_promo,0.235439,0.216127,0.367004,0.116301,1.000000,0.084955,-0.075321,0.020148,-0.066610,-0.000322,-0.071472,-0.004871,-0.005659,-0.001400,0.095823
google_mobility,0.058781,-0.213534,0.056074,0.073260,0.084955,1.000000,-0.756752,0.078227,-0.108645,0.003461,-0.546968,-0.031139,-0.043716,-0.014956,0.004846
covid_flag,-0.085571,0.281537,-0.024732,-0.094373,-0.075321,-0.756752,1.000000,0.007924,0.007924,-0.003052,0.722783,0.038692,0.052149,0.016564,-0.013360
v_day,-0.008442,-0.044565,0.019761,-0.044805,0.020148,0.078227,0.007924,1.000000,-0.020101,-0.019669,-0.002840,0.087736,0.074717,0.091110,0.001025
easter,-0.010266,0.000959,0.019761,-0.044805,-0.066610,-0.108645,0.007924,-0.020101,1.000000,-0.019669,-0.002840,0.098093,0.084211,0.091110,0.001025
christmas,-0.035916,-0.012153,0.013799,0.045834,-0.000322,0.003461,-0.003052,-0.019669,-0.019669,1.000000,-0.010954,-0.099654,-0.100675,-0.111085,-0.007852


In [ ]:
px.imshow(df.corr().to_pandas(), text_auto=True, aspect="auto")

In [13]:
df.groupby(['product']).agg(total_sales_month = ('date','count')).reset_index()

,product,total_sales_month
0,SKU3,204
1,SKU4,204
2,SKU1,204
3,SKU5,204
4,SKU6,198
5,SKU2,204


Note: SKU6 has 6 missing values

## Plots

In [ ]:
df.groupby(['product', 'year', 'month']).agg(total_sales_month = ('sales','sum')).reset_index()

,product,year,month,total_sales_month
0,SKU1,2019,5,112679
1,SKU5,2020,3,45005
2,SKU6,2019,5,134776
3,SKU4,2018,7,55910
4,SKU4,2019,10,111436
5,SKU1,2020,10,125405
6,SKU2,2017,11,13505
7,SKU3,2020,6,143807
8,SKU2,2017,7,22825
9,SKU6,2020,9,169003


In [ ]:
df.groupby(['product', 'cw']).agg(total_sales_week = ('sales','sum')).reset_index().plot.scatter(x='cw', y='total_sales_week', 
color="product", title="Total Weekly sales over 2017-2020")

In [ ]:
df.groupby(['product', 'date']).agg(mean_sales = ('sales','mean')).reset_index().plot.scatter(x='date', y='mean_sales', color="product", title="Mean sales over 2017-2020")

In [ ]:
df[df['product'] == 'SKU1'].groupby(['year','cw']).agg(mean_sales = ('sales','mean')).reset_index().plot.scatter(x='cw', y='mean_sales', color="year", title="Mean sales over 2017-2020")